In [1]:
!pip install azure-cli

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.9/439.9 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.7/291.7 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.9/365.9 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 546.8/546.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
from pathlib import Path
from dotenv import set_key,get_key,find_dotenv

In [3]:
subscripition_id = 'd5a877de-4915-4b9c-ab6b-a05ae383f23a'
resource_group = 'kubegroup'
location = 'eastus'
agent_size = 'Standard_DS2_v2'
aks_name = 'kubeaks'
agent_count = 2 
storage_account = 'storagelab'
storage_container = 'blobfuse'

In [4]:
env_path = find_dotenv()
if env_path == "":
    Path('.env').touch()
    env_path = find_dotenv()

NameError: name 'find_dotenv' is not defined

In [ ]:
set_key(env_path,'subscripition_id',subscripition_id)
set_key(env_path,'resource_group',resource_group)
set_key(env_path,'storage_container',storage_container)
set_key(env_path,'storage_account',storage_account)

(True, 'storage_account', 'storagelab')

In [30]:
!az login

A web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "a386e66e-6736-49ca-be72-6ce35f212f73",
    "id": "d5a877de-4915-4b9c-ab6b-a05ae383f23a",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Azure subscription 1",
    "state": "Enabled",
    "tenantId": "a386e66e-6736-49ca-be72-6ce35f212f73",
    "user": {
      "name": "kiqeng@tekiq.onmicrosoft.com",
      "type": "user"
    }
  }
]


In [31]:
!az account set -s {subscripition_id}

In [32]:
!az group create --name {resource_group} --location {location}

{
  "id": "/subscriptions/d5a877de-4915-4b9c-ab6b-a05ae383f23a/resourceGroups/kubegroup",
  "location": "eastus",
  "managedBy": null,
  "name": "kubegroup",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


In [33]:
!az aks create --node-vm-size {agent_size} --resource-group {resource_group} --name {aks_name} --node-count {agent_count} --kubernetes-version=1.22.6 --generate-ssh-keys --query 'provisioningState'

SSH key files '/home/kiq/.ssh/id_rsa' and '/home/kiq/.ssh/id_rsa.pub' have been generated under ~/.ssh to allow SSH access to the VM. If using machines without permanent storage like Azure Cloud Shell without an attached file share, back up your keys to a safe location
Resource provider 'Microsoft.ContainerService' used by this operation is not registered. We are registering for you.
Registration succeeded.
"Succeeded" ..


In [1]:
!sudo az aks install-cli

[sudo] password for kiq: 


In [5]:
!az aks get-credentials --resource-group {resource_group} --name {aks_name}

Merged "kubeaks" as current context in /home/kiq/.kube/config


In [6]:
!kubectl get nodes

NAME                                STATUS   ROLES   AGE     VERSION
aks-nodepool1-31663731-vmss000000   Ready    agent   8m35s   v1.22.6
aks-nodepool1-31663731-vmss000001   Ready    agent   8m34s   v1.22.6


In [7]:
node_names = !kubectl get nodes -o name
!kubectl describe node {node_names[0].strip('node/')}

Name:               aks-nodepool1-31663731-vmss000000
Roles:              agent
Labels:             agentpool=nodepool1
                    beta.kubernetes.io/arch=amd64
                    beta.kubernetes.io/instance-type=Standard_DS2_v2
                    beta.kubernetes.io/os=linux
                    failure-domain.beta.kubernetes.io/region=eastus
                    failure-domain.beta.kubernetes.io/zone=0
                    kubernetes.azure.com/agentpool=nodepool1
                    kubernetes.azure.com/cluster=MC_kubegroup_kubeaks_eastus
                    kubernetes.azure.com/kubelet-identity-client-id=26fe4521-d48c-4740-b12a-263063ed4fd5
                    kubernetes.azure.com/mode=system
                    kubernetes.azure.com/node-image-version=AKSUbuntu-1804gen2containerd-2022.07.11
                    kubernetes.azure.com/os-sku=Ubuntu
                    kubernetes.azure.com/role=agent
                    kubernetes.azure.com/storageprofile=managed
                 

In [9]:
!kubectl create -f https://raw.githubusercontent.com/NVIDIA/k8s-device-plugin/v1.11/nvidia-device-plugin.yml

error: resource mapping not found for name: "nvidia-device-plugin-daemonset" namespace: "kube-system" from "https://raw.githubusercontent.com/NVIDIA/k8s-device-plugin/v1.11/nvidia-device-plugin.yml": no matches for kind "DaemonSet" in version "extensions/v1beta1"
ensure CRDs are installed first


In [10]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                  READY   STATUS    RESTARTS   AGE
kube-system   azure-ip-masq-agent-rwcjw             1/1     Running   0          13m
kube-system   azure-ip-masq-agent-xsnxl             1/1     Running   0          13m
kube-system   cloud-node-manager-94bjs              1/1     Running   0          13m
kube-system   cloud-node-manager-nktk6              1/1     Running   0          13m
kube-system   coredns-autoscaler-7d56cd888-8lnb4    1/1     Running   0          14m
kube-system   coredns-dc97c5f55-92bqr               1/1     Running   0          14m
kube-system   coredns-dc97c5f55-kqz98               1/1     Running   0          12m
kube-system   csi-azuredisk-node-kbnsl              3/3     Running   0          13m
kube-system   csi-azuredisk-node-tds8c              3/3     Running   0          13m
kube-system   csi-azurefile-node-d84h8              3/3     Running   0          13m
kube-system   csi-azurefile-node-tkxcj              3/3     Runni